In [14]:
import paramiko
import os
import re
import requests
import json
import time

In [30]:
def run_in_master(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("namenode", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"cd /app/ && . /env_var_path.sh && {command}")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

def merge_results(path):
    run_in_master(f"hdfs dfs -cat {path}/part-* | hdfs dfs -put - {path}/merged.txt")
    
def get_data_from_output_path(path):
    return f"{path}/merged.txt"

def print_hdfs_output(path):
    raw = run_in_master(f"hdfs dfs -cat {get_data_from_output_path(path)}")[0]
    print("\n".join(raw[0:1000]))

def get_time(res, max_attempts: int = 4):
    def get_id(res):
        for line in res[1]:
            m = re.search('tracking URL: http://resourcemanager:8088/proxy/(.*)/', line)
            if m != None and m.group(1) != '':
                return m.group(1)
        return ''

    def get_time_from_data(data):
        sum = 0
        for attemp in data['attempts']:
            if attemp['completed'] is False:
                print('spark history server has not updated (yet)')
                return -1
            sum = sum + attemp['duration']
        return sum
        
    id = get_id(res)
    if id == -1:
        return -1

    attempt = 0

    while attempt < max_attempts:
        attempt = attempt + 1
        response = requests.get(f'http://namenode:18080/api/v1/applications/{id}')
        if not response.ok:
            print('WARNING: application error')
            return -1
            
        data = json.loads(response.text)
        t = get_time_from_data(data)

        if t >= 0:
            return t
        time.sleep(3)
    print('WARNING: maximum attempts exceeded')
    return -1

In [ ]:
# Tutaj odpalenie pythona
# Potrzebna akwizycja wcześniej! (dane covid)
run_in_master(f"hdfs dfs -rm -r /spark-result")
res01 = run_in_master("spark-submit --master yarn --deploy-mode cluster /data/master_volume/spark_scripts/test.py")
_ = merge_results("/spark-result/sql-select")
print(get_time(res01))
print_hdfs_output("/spark-result/sql-select")

In [27]:
get_time(res01)

19492

In [4]:
# Tutaj jak odpalić jarke, jakby było trzeba. Scala 2.13 jest potrzebna.
run_in_master("spark-submit --master yarn --deploy-mode cluster --class org.apache.spark.examples.SparkPi $SPARK_HOME/examples/jars/spark-examples_2.13-3.4.0.jar 10")

([],
 ['23/06/17 15:35:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable\n',
  '23/06/17 15:35:54 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at resourcemanager/172.25.0.7:8032\n',
  '23/06/17 15:35:54 INFO AHSProxy: Connecting to Application History server at historyserver/172.25.0.5:10200\n',
  '23/06/17 15:35:54 INFO Configuration: resource-types.xml not found\n',
  "23/06/17 15:35:54 INFO ResourceUtils: Unable to find 'resource-types.xml'.\n",
  '23/06/17 15:35:54 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (4096 MB per container)\n',
  '23/06/17 15:35:54 INFO Client: Will allocate AM container, with 1408 MB memory including 384 MB overhead\n',
  '23/06/17 15:35:54 INFO Client: Setting up container launch context for our AM\n',
  '23/06/17 15:35:54 INFO Client: Setting up the launch environment for our AM contai